In [ ]:
# Full notebook script: baseline preserved + added cleaning, viz, outlier analysis, tuning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, log_loss
from scipy import stats
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# ==========================================
# 1. USER CONFIGURATION (EDIT THIS PART ONLY)
# ==========================================
TRAIN_PATH = "/kaggle/input/ai-201-b-mse-2-aiml-c/train.csv"
TEST_PATH = "/kaggle/input/ai-201-b-mse-2-aiml-c/test.csv"
TARGET_COL = "NObeyesdad"
ID_COL = "id"
OUTPUT_FILE = "submission.csv"
# Toggle heavy steps (set False to skip long operations)
DO_PLOTTING = True
DO_OUTLIER_CAP = False  # If True, numeric outliers will be capped using IQR method
DO_HYPERPARAM_TUNING = True  # If True, run RandomizedSearchCV (can be slow)
RANDOM_STATE = 42
# ==========================================

# 2. Load Data
print("Loading data...")
train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)

print(f"Train shape: {train_data.shape}")
print(f"Test shape: {test_data.shape}")

# Keep test IDs for submission
test_ids = test_data[ID_COL] if ID_COL in test_data.columns else None

# Drop ID cols from feature tables to match previous logic
if ID_COL in train_data.columns:
    train_data = train_data.drop(columns=[ID_COL])
if ID_COL in test_data.columns:
    test_data = test_data.drop(columns=[ID_COL])

# -------------------------
# DATA CLEANING (10 marks)
# -------------------------
print("\n=== Data Cleaning ===")

# 1) Basic info
print("\nTrain info:")
print(train_data.info())
print("\nTest info:")
print(test_data.info())

# 2) Duplicates
train_dups = train_data.duplicated().sum()
print(f"\nDuplicate rows in train: {train_dups}")
if train_dups > 0:
    print("Dropping duplicate rows from train.")
    train_data = train_data.drop_duplicates().reset_index(drop=True)

# 3) Missing value summary
print("\nMissing values (train):")
print(train_data.isnull().sum()[train_data.isnull().sum() > 0])
print("\nMissing values (test):")
print(test_data.isnull().sum()[test_data.isnull().sum() > 0])

# 4) Inconsistent categorical values (basic check)
cat_columns_guess = train_data.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"\nDetected categorical columns for checking consistency: {cat_columns_guess}")
for c in cat_columns_guess:
    unique_vals = train_data[c].unique()
    if len(unique_vals) <= 20:
        print(f" Column `{c}` unique values: {unique_vals}")

# 5) Target distribution check
if TARGET_COL in train_data.columns:
    print("\nTarget distribution:")
    print(train_data[TARGET_COL].value_counts(normalize=True))
else:
    raise ValueError(f"Target column {TARGET_COL} not found in train data.")

# Keep a copy for visualization without destructive changes
train_viz = train_data.copy()

# -------------------------
# PREPROCESSING (kept as logic)
# -------------------------
# Separate X and y as before (preserve original logic)
X = train_data.drop(columns=[TARGET_COL])
y = train_data[TARGET_COL]

# Dynamic feature detection (same as original)
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['number']).columns.tolist()

print(f"\nDetected {len(cat_cols)} categorical columns: {cat_cols}")
print(f"Detected {len(num_cols)} numerical columns: {num_cols}")

# -------------------------
# OUTLIER ANALYSIS & HANDLING (10 marks)
# -------------------------
print("\n=== Outlier Analysis ===")
if DO_PLOTTING:
    # Histograms for numeric columns (small figure for speed; comment/uncomment as needed)
    n_num = len(num_cols)
    if n_num > 0:
        ncols = 3
        nrows = (n_num + ncols - 1) // ncols
        plt.figure(figsize=(5 * ncols, 4 * nrows))
        for i, col in enumerate(num_cols, 1):
            plt.subplot(nrows, ncols, i)
            sns.histplot(train_viz[col].dropna(), kde=True)
            plt.title(col)
        plt.tight_layout()
        plt.show()

    # Boxplots to visualize outliers
    if n_num > 0:
        plt.figure(figsize=(5 * ncols, 4 * nrows))
        for i, col in enumerate(num_cols, 1):
            plt.subplot(nrows, ncols, i)
            sns.boxplot(x=train_viz[col].dropna())
            plt.title(f"Boxplot: {col}")
        plt.tight_layout()
        plt.show()

    # Correlation heatmap for numerical features
    if n_num > 1:
        plt.figure(figsize=(10, 8))
        corr = train_viz[num_cols].corr()
        sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm")
        plt.title("Correlation matrix (numeric features)")
        plt.show()

    # Countplots for categorical features (top categories only)
    for c in cat_cols:
        plt.figure(figsize=(6, 4))
        sns.countplot(y=c, data=train_viz, order=train_viz[c].value_counts().index[:20])
        plt.title(f"Counts for {c}")
        plt.tight_layout()
        plt.show()

# -------------- IQR based outlier detection (non-destructive)
outlier_summary = {}
for col in num_cols:
    q1 = train_viz[col].quantile(0.25)
    q3 = train_viz[col].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    n_lower = (train_viz[col] < lower).sum()
    n_upper = (train_viz[col] > upper).sum()
    outlier_summary[col] = {"lower": lower, "upper": upper, "n_lower": int(n_lower), "n_upper": int(n_upper)}
print("\nOutlier summary (IQR method):")
for col, s in outlier_summary.items():
    print(f" {col}: below {s['n_lower']}, above {s['n_upper']}")

# Optionally cap outliers (explicitly controlled by DO_OUTLIER_CAP)
if DO_OUTLIER_CAP:
    print("\nCapping numeric outliers using IQR thresholds (applied to both train/test).")
    for col, s in outlier_summary.items():
        lower, upper = s['lower'], s['upper']
        # Cap in X (train features) and in test_data numeric cols
        X[col] = X[col].clip(lower=lower, upper=upper)
        if col in test_data.columns:
            test_data[col] = test_data[col].clip(lower=lower, upper=upper)

# -------------------------
# MISSING VALUES IMPUTATION (kept logic but documented)
# -------------------------
print("\n=== Missing Value Imputation ===")
# Numeric imputation
if num_cols:
    mean_vals = X[num_cols].mean()
    print("Numeric mean fill (per column):")
    print(mean_vals)
    X[num_cols] = X[num_cols].fillna(mean_vals)
    test_data[num_cols] = test_data[num_cols].fillna(mean_vals)

# Categorical imputation
if cat_cols:
    mode_vals = X[cat_cols].mode().iloc[0]
    print("Categorical mode fill (per column):")
    print(mode_vals.to_dict())
    X[cat_cols] = X[cat_cols].fillna(mode_vals)
    test_data[cat_cols] = test_data[cat_cols].fillna(mode_vals)

# -------------------------
# VISUAL CHECK AFTER IMPUTATION
# -------------------------
print("\nAfter imputation, missing values (train):")
print(X.isnull().sum()[X.isnull().sum() > 0])

# -------------------------
# DEFINE PREPROCESSOR (kept logic)
# -------------------------
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), cat_cols),
    ('num', StandardScaler(), num_cols)
])

# -------------------------
# TRAIN/VALIDATION SPLIT (kept logic)
# -------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3,
                                                  random_state=RANDOM_STATE, stratify=y)

# Fit preprocessor (kept)
print("\nTransforming data (fit preprocessor on train)...")
X_train_pre = preprocessor.fit_transform(X_train)
X_val_pre = preprocessor.transform(X_val)
test_data_pre = preprocessor.transform(test_data)

# -------------------------
# LABEL ENCODING (kept)
# -------------------------
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc = le.transform(y_val)

# -------------------------
# BASELINE MODEL (kept)
# -------------------------
print("\n=== Baseline RandomForest Training ===")
baseline_rfc = RandomForestClassifier(n_estimators=1000, random_state=RANDOM_STATE,
                                      class_weight='balanced', n_jobs=-1)
baseline_rfc.fit(X_train_pre, y_train_enc)

# Baseline predictions
train_proba_base = baseline_rfc.predict_proba(X_train_pre)
val_proba_base = baseline_rfc.predict_proba(X_val_pre)

# Metrics for baseline
n_classes_base = train_proba_base.shape[1]
if n_classes_base == 2:
    roc_train_base = roc_auc_score(y_train_enc, train_proba_base[:, 1])
    roc_val_base = roc_auc_score(y_val_enc, val_proba_base[:, 1])
else:
    roc_train_base = roc_auc_score(y_train_enc, train_proba_base, multi_class='ovr', average='macro')
    roc_val_base = roc_auc_score(y_val_enc, val_proba_base, multi_class='ovr', average='macro')

loss_train_base = log_loss(y_train_enc, train_proba_base)
loss_val_base = log_loss(y_val_enc, val_proba_base)

print(f"Baseline Training ROC AUC: {roc_train_base:.4f}")
print(f"Baseline Validation ROC AUC: {roc_val_base:.4f}")
print(f"Baseline Training Log Loss: {loss_train_base:.4f}")
print(f"Baseline Validation Log Loss: {loss_val_base:.4f}")

# -------------------------
# HYPERPARAMETER TUNING (10 marks)
# -------------------------
tuned_rfc = None
if DO_HYPERPARAM_TUNING:
    print("\n=== Hyperparameter Tuning (RandomizedSearchCV) ===")
    # Parameter grid for RandomizedSearch (typical ranges for RF)
    param_dist = {
        "n_estimators": [200, 500, 800, 1000],
        "max_depth": [None, 6, 10, 15, 20],
        "min_samples_split": [2, 5, 8, 10],
        "min_samples_leaf": [1, 2, 4, 6],
        "max_features": ["auto", "sqrt", 0.2, 0.5]
    }

    rnd_search = RandomizedSearchCV(
        estimator=RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced', n_jobs=-1),
        param_distributions=param_dist,
        n_iter=25,
        scoring='roc_auc_ovr' if n_classes_base > 2 else 'roc_auc',
        cv=3,
        verbose=1,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    rnd_search.fit(X_train_pre, y_train_enc)
    print("Best params found:")
    print(rnd_search.best_params_)
    print(f"Best CV score: {rnd_search.best_score_:.4f}")

    tuned_rfc = rnd_search.best_estimator_

    # Evaluate tuned model on train & val sets
    train_proba_tuned = tuned_rfc.predict_proba(X_train_pre)
    val_proba_tuned = tuned_rfc.predict_proba(X_val_pre)

    if n_classes_base == 2:
        roc_train_tuned = roc_auc_score(y_train_enc, train_proba_tuned[:, 1])
        roc_val_tuned = roc_auc_score(y_val_enc, val_proba_tuned[:, 1])
    else:
        roc_train_tuned = roc_auc_score(y_train_enc, train_proba_tuned, multi_class='ovr', average='macro')
        roc_val_tuned = roc_auc_score(y_val_enc, val_proba_tuned, multi_class='ovr', average='macro')

    loss_train_tuned = log_loss(y_train_enc, train_proba_tuned)
    loss_val_tuned = log_loss(y_val_enc, val_proba_tuned)

    print("\nTuned model metrics:")
    print(f"Tuned Training ROC AUC: {roc_train_tuned:.4f}")
    print(f"Tuned Validation ROC AUC: {roc_val_tuned:.4f}")
    print(f"Tuned Training Log Loss: {loss_train_tuned:.4f}")
    print(f"Tuned Validation Log Loss: {loss_val_tuned:.4f}")
else:
    print("\nHyperparameter tuning skipped by configuration.")

# -------------------------
# OPTION: choose model for submission
# We preserve your original logic: baseline_rfc is the trained model used by default.
# If you want to use the tuned model for submission, set `use_tuned=True` below.
# -------------------------
use_tuned = True if (DO_HYPERPARAM_TUNING and tuned_rfc is not None) else False
model_for_submission = tuned_rfc if use_tuned else baseline_rfc
if use_tuned:
    print("\nUsing TUNED model for submission.")
else:
    print("\nUsing BASELINE model for submission (same as original logic).")

# -------------------------
# OPTIONAL: ROC CURVE PLOT (small)
# -------------------------
if DO_PLOTTING:
    try:
        from sklearn.metrics import RocCurveDisplay
        plt.figure(figsize=(6, 6))
        if n_classes_base == 2:
            RocCurveDisplay.from_estimator(model_for_submission, X_val_pre, y_val_enc)
            plt.title("ROC Curve (Validation)")
            plt.show()
        else:
            # Plot per-class ROC curves for multiclass
            y_val_binarized = pd.get_dummies(y_val_enc)
            # compute per-class ROC curves if desired (skipping heavy code here)
            print("Multiclass ROC curve plotting skipped to keep notebook simple.")
    except Exception as e:
        print("Could not plot ROC curve:", e)

# -------------------------
# GENERATE SUBMISSION (kept logic)
# -------------------------
print("\n=== Generating submission ===")
test_pred_enc = model_for_submission.predict(test_data_pre)
test_pred = le.inverse_transform(test_pred_enc)

if test_ids is not None:
    submission_df = pd.DataFrame({
        ID_COL: test_ids,
        TARGET_COL: test_pred
    })
else:
    submission_df = pd.DataFrame({TARGET_COL: test_pred})

submission_df.to_csv(OUTPUT_FILE, index=False)
print(f"Submission saved to {OUTPUT_FILE}")
print(submission_df.head())

# -------------------------
# FINAL NOTES (for your rubric)
# -------------------------
print("\n=== Rubric mapping summary ===")
print("Data Cleaning and Preprocessing: ADDED (duplicates, missing, imputation, documented).")
print("Data Visualization and Outlier Analysis: ADDED (histograms, boxplots, corr heatmap, IQR summary).")
print("Model Training: Present (baseline RF as before).")
print("Hyperparameter Tuning: ADDED (RandomizedSearchCV) — optional and compared to baseline.")
print("Kaggle Submission: Preserved (same format and logic as your original script).")
